## 패턴/선호도 분석
1. 사용자별 방문 패턴 분석: 각 사용자가 방문한 페이지별로 평균 세션 지속 시간을 계산하고, 각 페이지별로 가장 긴 평균 세션 시간을 가진 사용자를 찾으세요. 이를 위해 각 사용자의 페이지별 평균 세션 시간을 계산하고, 각 페이지에서 가장 긴 평균 세션 시간을 모두 더하고 정수형으로 출력하시오.

2. 시간대별 페이지 선호도 분석: 하루를 네 시간대로 나누고 (새벽: 0-6시, 오전: 6-12시, 오후: 12-18시, 저녁: 18-24시), 각 시간대별로 가장 많이 방문된 페이지를 찾으세요. 각 시간대별로 가장 많이 방문된 페이지의 이름과 해당 시간대의 방문 횟수를 찾으세요! (단 0-6시 일때 0시부터 6시 전까지입니다.)
- 2-1: 시간대 별로 나누었을 때 가장 많이 방문한 페이지의 이름을 구하시오.
- 2-2: 시간대 별로 나누었을 때 가장 방문 횟수가 큰 값을 구하시오 

3. 재방문 패턴 분석: 사용자가 같은 날짜에 여러 페이지를 방문하는 경우를 '재방문'으로 간주합니다. 재방문한 사용자들의 데이터를 분석하여, 재방문한 날짜별 총 방문 페이지 수를 구하고 가장 재방문이 많은 월을 정수로 구하시오 

In [1]:
import pandas as pd
df = pd.read_csv("dataset/website.csv")

In [2]:
df.head()

,UserID,StartTime,EndTime,Page
0,1,2023-07-02 08:32:00,2023-07-02 12:41:00,Page1
1,1,2023-05-16 11:37:00,2023-05-16 13:29:00,Page5
2,1,2023-05-05 03:39:00,2023-05-05 05:48:00,Page2
3,1,2023-05-04 23:04:00,2023-05-05 01:54:00,Page3
4,1,2023-09-04 11:27:00,2023-09-04 14:09:00,Page1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601 entries, 0 to 9600
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   UserID     9601 non-null   int64 
 1   StartTime  9601 non-null   object
 2   EndTime    9601 non-null   object
 3   Page       9601 non-null   object
dtypes: int64(1), object(3)
memory usage: 300.2+ KB


In [4]:
df['StartTime'] = pd.to_datetime(df['StartTime'])
df['EndTime'] = pd.to_datetime(df['EndTime'])

In [5]:
# 1. 사용자별 방문 패턴 분석: 각 사용자가 방문한 페이지별로 평균 세션 지속 시간을 계산하고, 각 페이지별로 가장 긴 평균 세션 시간을 가진 사용자를 찾으세요.
# 이를 위해 각 사용자의 페이지별 평균 세션 시간을 계산하고, 각 페이지에서 가장 긴 평균 세션 시간을 모두 더하고 정수형으로 출력하시오.

In [6]:
df['StartTime'][0]

Timestamp('2023-07-02 08:32:00')

In [7]:
df['Duration'] = (df['EndTime'] - df['StartTime']).dt.total_seconds() / 3600

In [8]:
int(df.groupby(by=['UserID', 'Page'], as_index=False)['Duration'].mean().groupby('Page')['Duration'].max().sum())

24

In [9]:
# 2. 시간대별 페이지 선호도 분석: 하루를 네 시간대로 나누고 (새벽: 0-6시, 오전: 6-12시, 오후: 12-18시, 저녁: 18-24시), 각 시간대별로 가장 많이 방문된 페이지를 찾으세요.
# 각 시간대별로 가장 많이 방문된 페이지의 이름과 해당 시간대의 방문 횟수를 찾으세요! (단 0-6시 일때 0시부터 6시 전까지입니다.)

In [10]:
# 2-1. 시간대 별로 나누었을 때 가장 많이 방문한 페이지의 이름을 구하시오.
def time_band(x):
    if 0 <= x < 6:
        x = '새벽'
    elif 6 <= x < 12:
        x = '오전'
    elif 12 <= x < 18:
        x = '오후'
    else:
        x = '저녁'
    return x

df['TimeBand'] = df['StartTime'].dt.hour.apply(time_band)
df.groupby(by=['TimeBand', 'Page'], as_index=False).size().max()['Page']

'Page5'

In [11]:
# 2-2: 시간대 별로 나누었을 때 가장 방문 횟수가 큰 값을 구하시오
df.groupby(by=['TimeBand', 'Page'], as_index=False).size().max()['size']

538

In [12]:
# 3. 재방문 패턴 분석: 사용자가 같은 날짜에 여러 페이지를 방문하는 경우를 '재방문'으로 간주합니다.
# 재방문한 사용자들의 데이터를 분석하여, 재방문한 날짜별 총 방문 페이지 수를 구하고 가장 재방문이 많은 월을 정수로 구하시오

In [13]:
df.head()

,UserID,StartTime,EndTime,Page,Duration,TimeBand
0,1,2023-07-02 08:32:00,2023-07-02 12:41:00,Page1,4.150000,오전
1,1,2023-05-16 11:37:00,2023-05-16 13:29:00,Page5,1.866667,오전
2,1,2023-05-05 03:39:00,2023-05-05 05:48:00,Page2,2.150000,새벽
3,1,2023-05-04 23:04:00,2023-05-05 01:54:00,Page3,2.833333,저녁
4,1,2023-09-04 11:27:00,2023-09-04 14:09:00,Page1,2.700000,오전


In [25]:
gdf = df.groupby(by=['UserID', df['StartTime'].dt.date], as_index=False).size()
gdf = gdf[gdf['size'] > 1]
gdf

,UserID,StartTime,size
57,6,2023-01-10,2
88,10,2023-04-10,2
224,26,2023-06-16,2
268,31,2023-06-04,2
271,31,2023-08-26,2
...,...,...,...
9193,1164,2023-09-11,2
9227,1168,2023-06-21,2
9343,1187,2023-07-23,2
9374,1190,2023-10-22,2


In [26]:
gdf['StartTime'] = pd.to_datetime(gdf['StartTime'])

In [30]:
gdf.groupby(by=gdf['StartTime'].dt.month).size().idxmax()

4